# Mini-projet n°3 - ViT vs CNN

L'objectif de ce mini-projet est de comparer les performances de deux architectures différentes pour de la classification d'image : les Vision Transformers (ViT) et les Convolutional Neural Networks (CNN).

## Explication de l'architecture Vision Transformers

Les transformers ont initialement étaient inventés pour des problèmes NLP mais on peut aussi l'utiliser -comme ici- pour de la classification d'images. 

# Notes que j'ia rpise lors de l'explication du prof d'aif

POur mettre en relation les différents patchs on va faire une matrice d'attention ds laulle il est ecrit ok tel patch a tel relation avec tel patch.
query= question : cette matrice est optimisée comme key et value

on a emvbedding et matrices d eprojections (q,v,k) entre la matrice et le token embeddé (apres embedding) et ca donne vecteurs et apres la fprmule de l'attention on peut avoir une matrice de relation entre les differents tokens (tokens=patch) - une heatmap-


l'encoder permet de faier une representation des patchs avec projections dans un espace latent. puis le decoder prédit

# Vrai truc en dessous:

On a d'abord le patch embedding: découpe de l'image en patchs non chevauchant (par exemple de taille 16x16x3), ce patch est ensuite aplati/ linéarisé en vecteur 1D que l'on nommera $patch_flat$. Ainsi, pour chaque patch on a un vecteur de taille fixe qu'on nommera ici $N$. Ce vecteur brut contient les valeurs du patch mais n’est pas encore dans l’espace de représentation du Transformer or le transformer travaille avec une dimension d'embedding fixe $D$, par exemple, pour Vit-B $D=768$. Pour ce faire, on applique une transformation linéaire qui se traduit par une multiplication entre le vecteur $patch_flat$ et une matrice de paramètres appris $W$ de taille $[N * D]$.
Cette projection sert à créer de nouveaux vecteurs de taille $D$ capablent de capturer les formes, les textures, les couleurs etc de manière optimale pour le transformer.

**Qu'est ce que la matrice $W$?**

La matrice $W$ transforme un patch brut en un embedding de dimension $D$. Chaque colonne correspond à un filtre ou feature que le modèle apprend à détecter dans un patch. Chaque ligne correspond à un pixel spécifique du patch. Cette matrice est entraîné par backpropagation lors de l'entraînement. 

Ainsi, la matrice $W$ apprend à produire des vecteurs qui contiennent exactement les informations visuelles utiles pour les couches Transformer qui suivent. Chaque valeur de ce nouveau vecteur après projection est appelé *activation*. On appellera par la suite ces nouveaux vecteurs: $patch_projected$.


Après tout cela, un *encodage positionnel* est effectué pour chaque patch afin de conserver la structure spatiale de l'image originale. Des encodages positionnels sont ajoutés à chaque embedding de patch afin d’injecter des informations sur la position des patchs dans l’image. Ces encodages permettent au modèle de capturer les relations spatiales entre les patchs, de manière analogue à la façon dont les Transformers en traitement du langage naturel (NLP) encodent la position des mots dans une phrase.
Dans les Vision Transformers, les encodages positionnels peuvent être soit appris pendant l’entraînement, soit prédéfinis (fixes). La plupart des implémentations utilisent des encodages positionnels appris, qui permettent au modèle d’adapter dynamiquement les représentations des positions pour la tâche spécifique.

Après ces étpes, les patchs sont incorporer dans *l'encodeur*. Celui-ci est composé de L blocs identiques.
Dans chaque block on retrouve le LayerNorm suivi d'un Multi-head attention puis d'un add & skip et ensuite de nouveau un LayerNorm suivi d'un Feed-Forward. Après ces étapes de blocs dans l'encodeur, on fait un embedding final et vient ensuite la classfication finale.

**Explication du LayerNorm**

Chaque *activation* $x_i$, donc les valeurs des vecteurs $patch_projected$ sont normalisées (soustraction de la moyenne et division de l'ecart type), on obteint donc $\hat{x_i}$. Puis, pour chaque valeur (activation), on ajoute un biais et un gain qui sont des paramètres appris par le modèle tel que : $\gamma * \hat{x_i} + \beta$. Ainsi, toutes les dimensions sont centrées autour de 0 et ont une variance similaire : les vecteurs sont “équilibrés”.
Cela permet au Transformer de mieux combiner les informations et d’éviter que certaines valeurs explosent ou disparaissent pendant l’entraînement.

**Explication du Multi-Head Attention**

Permet de capturer les relations entre tokens (vecteurs correspondant un patch) **A faire**

**Explication du Add & Skip**

On ajoute l'entrée d'une sous-couche à sa sortie, c'est à dire que l'entrée de LayerNorm est ajoutée à la sortie de Multi-head Attention. Même chose apres le Feed-Forward. Cela sert à ce que le bloc n’ait à apprendre que la différence par rapport à l’entrée, pour en pas tout recalculer. de plus, dans les réseaux profonds, il est possible d'avoir des vanishing gardients. Ajouter l'entrée assure que le gradient peut toujours circuler directement de la sortie vers l’entrée.

En résumé, le Add & Skip sert à rendre l’apprentissage plus stable et efficace, en laissant passer l’information originale et en ajoutant seulement ce que la sous-couche apprend de nouveau.

**Explication du Feed-Forward**

Ici chaque token est traité individuellement. Le FFn est un MLP appliqué séparément à chaque token. de manièrte plus détaillée, le MLP va projeté le vecteur dans une dimension plus grande afin de crée plus de liberté pour combiner et recombiner les informations. Puis il va appliquer une fonction d'activation non linéaire GELU ce qui permet au réseau de modéliser des transformations complexes. Enfin, il va projetter de nouveaux les vecteurs dans la dimension de base $D$ pour que le Add & Skip foncftionne correctement.


Après l'encodeur, les différents vecteurs sont passés dans une couche linéaire pour produire les logits de classification. Ces logits sont ensuite passé dans une softmax pour faire la prédiction de classe.